# Update Script
This script will update the nba_db on our postgres server. <br>Looks at our last attempted pull date and pulls every date from then until the current date.

In [1]:
import io
import os
import psycopg2 as pg
from psycopg2 import Error
from sqlalchemy import create_engine
import pandas as pd
import datetime as dt 
import random
from time import sleep

 
import nba_api.stats.endpoints as ep

#internal scripts
import proj_functions as pf
from connection_data import c_data


### File Locations

In [2]:
loc_scoreboard = '/media/dan/Dan External/nba_data/scoreboard/'
loc_player_bs_trad = '/media/dan/Dan External/nba_data/player_bs_trad/'
loc_player_bs_advanced = '/media/dan/Dan External/nba_data/player_bs_advanced/'
loc_player_bs_ff = '/media/dan/Dan External/nba_data/player_bs_ff/'
loc_player_bs_misc = '/media/dan/Dan External/nba_data/player_bs_misc/'
loc_player_bs_pt = '/media/dan/Dan External/nba_data/player_bs_pt/'
loc_player_bs_scoring = '/media/dan/Dan External/nba_data/player_bs_scoring/'
loc_player_bs_summ = '/media/dan/Dan External/nba_data/player_bs_summ/'
loc_player_bs_tracking = '/media/dan/Dan External/nba_data/player_bs_tracking/'
loc_player_bs_usage = '/media/dan/Dan External/nba_data/player_bs_usage/'

loc_team_bs_trad = '/media/dan/Dan External/nba_data/team_bs_trad/'
loc_team_bs_advanced = '/media/dan/Dan External/nba_data/team_bs_advanced/'
loc_team_bs_ff = '/media/dan/Dan External/nba_data/team_bs_ff/'
loc_team_bs_misc = '/media/dan/Dan External/nba_data/team_bs_misc/'
loc_team_bs_pt = '/media/dan/Dan External/nba_data/team_bs_pt/'
loc_team_bs_scoring = '/media/dan/Dan External/nba_data/team_bs_scoring/'
loc_team_bs_summ = '/media/dan/Dan External/nba_data/team_bs_summ/'
loc_team_bs_tracking = '/media/dan/Dan External/nba_data/team_bs_tracking/'

In [3]:
file_names = ['scoreboard_{}_{}_{}', 'player_bs_trad_{}_{}_{}', 'player_bs_adv_{}_{}_{}',
             'player_bs_ff_{}_{}_{}', 'player_bs_misc_{}_{}_{}', 'player_bs_pt_{}_{}_{}',
             'player_bs_scoring_{}_{}_{}', 'player_bs_summ_{}_{}_{}', 'player_bs_tracking_{}_{}_{}',
             'player_bs_usage_{}_{}_{}', 'team_bs_trad_{}_{}_{}', 'team_bs_adv_{}_{}_{}', 
             'team_bs_ff_{}_{}_{}', 'team_bs_misc_{}_{}_{}', 'team_bs_pt_{}_{}_{}', 
             'team_bs_scoring_{}_{}_{}', 'team_bs_summ_{}_{}_{}', 'team_bs_tracking_{}_{}_{}']
table_names = ['scoreboard', 'player_bs', 'player_bs_adv', 'player_bs_ff',
               'player_bs_misc', 'player_bs_pt', 'player_bs_scoring', 'player_bs_summ',
               'player_bs_tracking', 'player_bs_usage', 'team_bs', 'team_bs_adv', 
               'team_bs_ff', 'team_bs_misc','team_bs_pt', 'team_bs_scoring', 
              'team_bs_summ', 'team_bs_tracking']
save_locations = [loc_scoreboard, loc_player_bs_trad, loc_player_bs_advanced,
                  loc_player_bs_ff, loc_player_bs_misc, loc_player_bs_pt, 
                  loc_player_bs_scoring, loc_player_bs_summ, loc_player_bs_tracking,
                  loc_player_bs_usage, loc_team_bs_trad, loc_team_bs_advanced,
                  loc_team_bs_ff, loc_team_bs_misc, loc_team_bs_pt, loc_team_bs_scoring,
                  loc_team_bs_summ, loc_team_bs_tracking]

## Script

In [ ]:
def pull_games():
    #Start with the last date pulled
    last_pull_date = pf.get_last_date()[0]
    yesterday = dt.date.today() - dt.timedelta(1)

    while last_pull_date < yesterday:
        current_pull_date = last_pull_date + dt.timedelta(1)
        try:
            #add sleep to reduce chances of being blocked
            sleep(random.choices([1, 2, 5], weights=[.58, .41, .01])[0] * random.uniform(.01, 3))
            # pull the scoreboard for the current pull date
            sb = pf.get_scoreboard(current_pull_date)
            if type(sb) == str:
                pf.update_last_date(current_pull_date)
                last_pull_date = pf.get_last_date()[0]
                pass
            else:
                gids = sb['GAME_ID']
                # get all dataframes
                df_player_bs_advanced = pf.get_player_bs_advanced(gids)
                df_team_bs_advanced = pf.get_team_bs_advanced(gids)

                df_player_bs_ff = pf.get_player_bs_ff(gids)
                df_team_bs_ff = pf.get_team_bs_ff(gids)

                df_player_bs_misc = pf.get_player_bs_misc(gids)
                df_team_bs_misc = pf.get_team_bs_misc(gids)

                df_player_bs_pt = pf.get_player_bs_pt(gids)
                df_team_bs_pt = pf.get_player_bs_pt(gids)

                #add a sleep to reduce chances of being blocked
                sleep(random.choices([1, 2, 5], weights=[.58, .41, .01])[0] * random.uniform(.01, 3))

                df_player_bs_scoring = pf.get_player_bs_scoring(gids)
                df_team_bs_scoring = pf.get_team_bs_scoring(gids)

                df_player_bs_summ = pf.get_player_bs_summ(gids)
                df_team_bs_summ = pf.get_team_bs_summ(gids)

                df_player_bs_trad = pf.get_player_bs_trad(gids)
                df_team_bs_trad = pf.get_team_bs_trad(gids)

                df_player_bs_tracking = pf.get_player_bs_tracking(gids)
                df_team_bs_tracking = pf.get_team_bs_tracking(gids)

                df_player_bs_usage = pf.get_player_bs_usage(gids)

                dfs = [sb, df_player_bs_trad, df_player_bs_advanced, df_player_bs_ff,
                      df_player_bs_misc, df_player_bs_pt, df_player_bs_scoring,
                      df_player_bs_summ, df_player_bs_tracking, df_team_bs_trad,
                      df_team_bs_advanced, df_team_bs_ff, df_team_bs_misc, df_team_bs_pt,
                      df_team_bs_scoring, df_team_bs_summ, df_team_bs_tracking]

                # Save Dataframes if have data and upload to database
                for df, name, table, location in zip(dfs, file_names, table_names, save_locations):
                    file_name = name.format(current_pull_date.year, current_pull_date.month, current_pull_date.day)
                    df['file_name'] = file_name
                    pf.save_data(df, file_name, location)
                    pf.upload_data(df, table)

                #update the last pull date
                pf.update_last_date(current_pull_date)
                last_pull_date = pf.get_last_date()[0]
        # if we get an error, we change the vpn connection and start again
        except:
            print('finding new vpn')
            os.system('sudo /home/dan/python_code/nba_project/python_scripts/new_connection.sh')
            sleep(5)
            pull_games()
            
pull_games()
        

finding new vpn
scoreboard_1952_2_3 saved to /media/dan/Dan External/nba_data/scoreboard/
player_bs_trad_1952_2_3 saved to /media/dan/Dan External/nba_data/player_bs_trad/
player_bs_adv_1952_2_3 saved to /media/dan/Dan External/nba_data/player_bs_advanced/
player_bs_ff_1952_2_3 saved to /media/dan/Dan External/nba_data/player_bs_ff/
player_bs_misc_1952_2_3 saved to /media/dan/Dan External/nba_data/player_bs_misc/
player_bs_pt_1952_2_3 saved to /media/dan/Dan External/nba_data/player_bs_pt/
player_bs_scoring_1952_2_3 saved to /media/dan/Dan External/nba_data/player_bs_scoring/
player_bs_summ_1952_2_3 saved to /media/dan/Dan External/nba_data/player_bs_summ/
player_bs_tracking_1952_2_3 saved to /media/dan/Dan External/nba_data/player_bs_tracking/
player_bs_usage_1952_2_3 saved to /media/dan/Dan External/nba_data/player_bs_usage/
team_bs_trad_1952_2_3 saved to /media/dan/Dan External/nba_data/team_bs_trad/
team_bs_adv_1952_2_3 saved to /media/dan/Dan External/nba_data/team_bs_advanced/
te

scoreboard_1952_2_9 saved to /media/dan/Dan External/nba_data/scoreboard/
player_bs_trad_1952_2_9 saved to /media/dan/Dan External/nba_data/player_bs_trad/
player_bs_adv_1952_2_9 saved to /media/dan/Dan External/nba_data/player_bs_advanced/
player_bs_ff_1952_2_9 saved to /media/dan/Dan External/nba_data/player_bs_ff/
player_bs_misc_1952_2_9 saved to /media/dan/Dan External/nba_data/player_bs_misc/
player_bs_pt_1952_2_9 saved to /media/dan/Dan External/nba_data/player_bs_pt/
player_bs_scoring_1952_2_9 saved to /media/dan/Dan External/nba_data/player_bs_scoring/
player_bs_summ_1952_2_9 saved to /media/dan/Dan External/nba_data/player_bs_summ/
player_bs_tracking_1952_2_9 saved to /media/dan/Dan External/nba_data/player_bs_tracking/
player_bs_usage_1952_2_9 saved to /media/dan/Dan External/nba_data/player_bs_usage/
team_bs_trad_1952_2_9 saved to /media/dan/Dan External/nba_data/team_bs_trad/
team_bs_adv_1952_2_9 saved to /media/dan/Dan External/nba_data/team_bs_advanced/
team_bs_ff_1952_2_

team_bs_summ_1952_2_14 saved to /media/dan/Dan External/nba_data/team_bs_summ/
scoreboard_1952_2_15 saved to /media/dan/Dan External/nba_data/scoreboard/
player_bs_trad_1952_2_15 saved to /media/dan/Dan External/nba_data/player_bs_trad/
player_bs_adv_1952_2_15 saved to /media/dan/Dan External/nba_data/player_bs_advanced/
player_bs_ff_1952_2_15 saved to /media/dan/Dan External/nba_data/player_bs_ff/
player_bs_misc_1952_2_15 saved to /media/dan/Dan External/nba_data/player_bs_misc/
player_bs_pt_1952_2_15 saved to /media/dan/Dan External/nba_data/player_bs_pt/
player_bs_scoring_1952_2_15 saved to /media/dan/Dan External/nba_data/player_bs_scoring/
player_bs_summ_1952_2_15 saved to /media/dan/Dan External/nba_data/player_bs_summ/
player_bs_tracking_1952_2_15 saved to /media/dan/Dan External/nba_data/player_bs_tracking/
player_bs_usage_1952_2_15 saved to /media/dan/Dan External/nba_data/player_bs_usage/
team_bs_trad_1952_2_15 saved to /media/dan/Dan External/nba_data/team_bs_trad/
team_bs_a

team_bs_summ_1952_2_20 saved to /media/dan/Dan External/nba_data/team_bs_summ/
scoreboard_1952_2_21 saved to /media/dan/Dan External/nba_data/scoreboard/
player_bs_trad_1952_2_21 saved to /media/dan/Dan External/nba_data/player_bs_trad/
player_bs_adv_1952_2_21 saved to /media/dan/Dan External/nba_data/player_bs_advanced/
player_bs_ff_1952_2_21 saved to /media/dan/Dan External/nba_data/player_bs_ff/
player_bs_misc_1952_2_21 saved to /media/dan/Dan External/nba_data/player_bs_misc/
player_bs_pt_1952_2_21 saved to /media/dan/Dan External/nba_data/player_bs_pt/
player_bs_scoring_1952_2_21 saved to /media/dan/Dan External/nba_data/player_bs_scoring/
player_bs_summ_1952_2_21 saved to /media/dan/Dan External/nba_data/player_bs_summ/
player_bs_tracking_1952_2_21 saved to /media/dan/Dan External/nba_data/player_bs_tracking/
player_bs_usage_1952_2_21 saved to /media/dan/Dan External/nba_data/player_bs_usage/
team_bs_trad_1952_2_21 saved to /media/dan/Dan External/nba_data/team_bs_trad/
team_bs_a

team_bs_summ_1952_2_26 saved to /media/dan/Dan External/nba_data/team_bs_summ/
scoreboard_1952_2_27 saved to /media/dan/Dan External/nba_data/scoreboard/
player_bs_trad_1952_2_27 saved to /media/dan/Dan External/nba_data/player_bs_trad/
player_bs_adv_1952_2_27 saved to /media/dan/Dan External/nba_data/player_bs_advanced/
player_bs_ff_1952_2_27 saved to /media/dan/Dan External/nba_data/player_bs_ff/
player_bs_misc_1952_2_27 saved to /media/dan/Dan External/nba_data/player_bs_misc/
player_bs_pt_1952_2_27 saved to /media/dan/Dan External/nba_data/player_bs_pt/
player_bs_scoring_1952_2_27 saved to /media/dan/Dan External/nba_data/player_bs_scoring/
player_bs_summ_1952_2_27 saved to /media/dan/Dan External/nba_data/player_bs_summ/
player_bs_tracking_1952_2_27 saved to /media/dan/Dan External/nba_data/player_bs_tracking/
player_bs_usage_1952_2_27 saved to /media/dan/Dan External/nba_data/player_bs_usage/
team_bs_trad_1952_2_27 saved to /media/dan/Dan External/nba_data/team_bs_trad/
team_bs_a

...

player_bs_adv_1969_10_31 saved to /media/dan/Dan External/nba_data/player_bs_advanced/
player_bs_ff_1969_10_31 saved to /media/dan/Dan External/nba_data/player_bs_ff/
player_bs_misc_1969_10_31 saved to /media/dan/Dan External/nba_data/player_bs_misc/
player_bs_pt_1969_10_31 saved to /media/dan/Dan External/nba_data/player_bs_pt/
player_bs_scoring_1969_10_31 saved to /media/dan/Dan External/nba_data/player_bs_scoring/
player_bs_summ_1969_10_31 saved to /media/dan/Dan External/nba_data/player_bs_summ/
player_bs_tracking_1969_10_31 saved to /media/dan/Dan External/nba_data/player_bs_tracking/
player_bs_usage_1969_10_31 saved to /media/dan/Dan External/nba_data/player_bs_usage/
team_bs_trad_1969_10_31 saved to /media/dan/Dan External/nba_data/team_bs_trad/
team_bs_adv_1969_10_31 saved to /media/dan/Dan External/nba_data/team_bs_advanced/
team_bs_ff_1969_10_31 saved to /media/dan/Dan External/nba_data/team_bs_ff/
team_bs_misc_1969_10_31 saved to /media/dan/Dan External/nba_data/team_bs_misc

player_bs_adv_1969_11_6 saved to /media/dan/Dan External/nba_data/player_bs_advanced/
player_bs_ff_1969_11_6 saved to /media/dan/Dan External/nba_data/player_bs_ff/
player_bs_misc_1969_11_6 saved to /media/dan/Dan External/nba_data/player_bs_misc/
player_bs_pt_1969_11_6 saved to /media/dan/Dan External/nba_data/player_bs_pt/
player_bs_scoring_1969_11_6 saved to /media/dan/Dan External/nba_data/player_bs_scoring/
player_bs_summ_1969_11_6 saved to /media/dan/Dan External/nba_data/player_bs_summ/
player_bs_tracking_1969_11_6 saved to /media/dan/Dan External/nba_data/player_bs_tracking/
player_bs_usage_1969_11_6 saved to /media/dan/Dan External/nba_data/player_bs_usage/
team_bs_trad_1969_11_6 saved to /media/dan/Dan External/nba_data/team_bs_trad/
team_bs_adv_1969_11_6 saved to /media/dan/Dan External/nba_data/team_bs_advanced/
team_bs_ff_1969_11_6 saved to /media/dan/Dan External/nba_data/team_bs_ff/
team_bs_misc_1969_11_6 saved to /media/dan/Dan External/nba_data/team_bs_misc/
team_bs_pt

scoreboard_1969_11_12 saved to /media/dan/Dan External/nba_data/scoreboard/
player_bs_trad_1969_11_12 saved to /media/dan/Dan External/nba_data/player_bs_trad/
player_bs_adv_1969_11_12 saved to /media/dan/Dan External/nba_data/player_bs_advanced/
player_bs_ff_1969_11_12 saved to /media/dan/Dan External/nba_data/player_bs_ff/
player_bs_misc_1969_11_12 saved to /media/dan/Dan External/nba_data/player_bs_misc/
player_bs_pt_1969_11_12 saved to /media/dan/Dan External/nba_data/player_bs_pt/
player_bs_scoring_1969_11_12 saved to /media/dan/Dan External/nba_data/player_bs_scoring/
player_bs_summ_1969_11_12 saved to /media/dan/Dan External/nba_data/player_bs_summ/
player_bs_tracking_1969_11_12 saved to /media/dan/Dan External/nba_data/player_bs_tracking/
player_bs_usage_1969_11_12 saved to /media/dan/Dan External/nba_data/player_bs_usage/
team_bs_trad_1969_11_12 saved to /media/dan/Dan External/nba_data/team_bs_trad/
team_bs_adv_1969_11_12 saved to /media/dan/Dan External/nba_data/team_bs_adva

team_bs_summ_1969_11_18 saved to /media/dan/Dan External/nba_data/team_bs_summ/
scoreboard_1969_11_19 saved to /media/dan/Dan External/nba_data/scoreboard/
player_bs_trad_1969_11_19 saved to /media/dan/Dan External/nba_data/player_bs_trad/
player_bs_adv_1969_11_19 saved to /media/dan/Dan External/nba_data/player_bs_advanced/
player_bs_ff_1969_11_19 saved to /media/dan/Dan External/nba_data/player_bs_ff/
player_bs_misc_1969_11_19 saved to /media/dan/Dan External/nba_data/player_bs_misc/
player_bs_pt_1969_11_19 saved to /media/dan/Dan External/nba_data/player_bs_pt/
player_bs_scoring_1969_11_19 saved to /media/dan/Dan External/nba_data/player_bs_scoring/
player_bs_summ_1969_11_19 saved to /media/dan/Dan External/nba_data/player_bs_summ/
player_bs_tracking_1969_11_19 saved to /media/dan/Dan External/nba_data/player_bs_tracking/
player_bs_usage_1969_11_19 saved to /media/dan/Dan External/nba_data/player_bs_usage/
team_bs_trad_1969_11_19 saved to /media/dan/Dan External/nba_data/team_bs_tra

team_bs_pt_1969_11_25 saved to /media/dan/Dan External/nba_data/team_bs_pt/
team_bs_scoring_1969_11_25 saved to /media/dan/Dan External/nba_data/team_bs_scoring/
team_bs_summ_1969_11_25 saved to /media/dan/Dan External/nba_data/team_bs_summ/
scoreboard_1969_11_26 saved to /media/dan/Dan External/nba_data/scoreboard/
player_bs_trad_1969_11_26 saved to /media/dan/Dan External/nba_data/player_bs_trad/
player_bs_adv_1969_11_26 saved to /media/dan/Dan External/nba_data/player_bs_advanced/
player_bs_ff_1969_11_26 saved to /media/dan/Dan External/nba_data/player_bs_ff/
player_bs_misc_1969_11_26 saved to /media/dan/Dan External/nba_data/player_bs_misc/
player_bs_pt_1969_11_26 saved to /media/dan/Dan External/nba_data/player_bs_pt/
player_bs_scoring_1969_11_26 saved to /media/dan/Dan External/nba_data/player_bs_scoring/
player_bs_summ_1969_11_26 saved to /media/dan/Dan External/nba_data/player_bs_summ/
player_bs_tracking_1969_11_26 saved to /media/dan/Dan External/nba_data/player_bs_tracking/
p